<a href="https://colab.research.google.com/github/trtrgfh/GlVYfAbQjtuwUHlZ/blob/main/Potential_Talents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score, f1_score, make_scorer
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df = pd.read_csv("/content/potential-talents.csv")
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [5]:
job_titles = df["job_title"].to_list()
keywords = ["Aspiring human resources", "Seeking human resources"]

combined_list = keywords + job_titles

## Preprocessing

## Bag-of-words

In [6]:
from nltk.stem import PorterStemmer        

In [7]:
CountVec = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
#transform
Count_data = CountVec.fit_transform(combined_list)

keyword_vectors = Count_data[:len(keywords)].toarray()
job_title_vectors = Count_data[len(keywords):].toarray()

# # Get the stems 
# stemmer = PorterStemmer() 
# words = CountVec.get_feature_names_out()
# stems = [] 
# for word in words:
#     stem_word = stemmer.stem(word)  
#     stems.append(stem_word) 
 
#create dataframe
# cv_df=pd.DataFrame(Count_data.toarray(),columns=stems)
# cv_df

In [16]:
# Compute cosine similarity between the keyword vectors and job title vectors
cv_cs = cosine_similarity(keyword_vectors, job_title_vectors)

# # Threshold for cosine similarity to consider a match
# similarity_threshold = 0.5

# # Find the indices of job titles with cosine similarity above the threshold
# similar_job_indices = [
#     index for index, similarity in enumerate(cosine_similarities[0]) if similarity > similarity_threshold
# ]

# # Get the similar job titles
# similar_job_titles = [job_titles[index] for index in similar_job_indices]

# print(similar_job_titles[:5])

In [17]:
cv_res = pd.DataFrame(cv_cs,columns=df['id']).mean(axis=0).to_frame()

In [18]:
cv_res.head(10)

,0
id,
1,0.416667
2,0.000000
3,0.721688
4,0.000000
5,0.000000
6,0.721688
7,0.545545
8,0.000000
9,0.545545


## Tf-Idf vectorizer

In [11]:
tf_idf_vec = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english')
 
tf_idf_data_smooth = tf_idf_vec.fit_transform(combined_list)

In [13]:
tfidf_cs = cosine_similarity(keyword_vectors, job_title_vectors)

In [19]:
tfidf_res = pd.DataFrame(cv_cs,columns=df['id']).mean(axis=0).to_frame()
tfidf_res.head(10)

,0
id,
1,0.416667
2,0.000000
3,0.721688
4,0.000000
5,0.000000
6,0.721688
7,0.545545
8,0.000000
9,0.545545


## Word2vec

## Cosine Similarity

In [9]:
def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''

    ### START CODE HERE ###
    dot = np.dot(A, B)    
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)    
    cos = dot / (norma * normb)

    ### END CODE HERE ###
    return cos

In [6]:
# def get_country(city1, country1, city2, embeddings, cosine_similarity=cosine_similarity):
#     """
#     Input:
#         city1: a string (the capital city of country1)
#         country1: a string (the country of capital1)
#         city2: a string (the capital city of country2)
#         embeddings: a dictionary where the keys are words and values are their emmbeddings
#     Output:
#         countries: a dictionary with the most likely country and its similarity score
#     """
#     group = (city1, country1, city2)
#     city1_emb = embeddings[city1]
#     country1_emb = embeddings[country1]
#     city2_emb = embeddings[city2]

#     # Remember: King - Man + Woman = Queen
#     vec = country1_emb - city1_emb + city2_emb

#     similarity = -1
#     country = 'Irap'

#     # loop through all words in the embeddings dictionary
#     for word in embeddings.keys():

#         # first check that the word is not already in the 'group'
#         if word not in group:

#             # get the word embedding
#             word_emb = embeddings[word]

#             # calculate cosine similarity between embedding of country 2 and the word in the embeddings dictionary
#             cur_similarity = cosine_similarity(vec, word_emb)

#             # if the cosine similarity is more similar than the previously best similarity...
#             if cur_similarity > similarity:

#                 # update the similarity to the new, better similarity
#                 similarity = cur_similarity

#                 # store the country as a tuple, which contains the word and the similarity
#                 country = (word, similarity)

#     ### END CODE HERE ###

#     return country